<a href="https://colab.research.google.com/github/AdnanMulla/ai-courses/blob/module-1/AI%20Agents%20and%20Agentic%20AI%20with%20Python%20%26%20Generative%20AI/Module%201/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'GEMINI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('GEMINI_API_KEY')
os.environ["GEMINI_API_KEY"] = api_key

Below is our first API interaction with LLM.

In [2]:
from litellm import completion
from typing import List, Dict
import json

print("START")

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
      model="gemini/gemini-2.0-flash",
      messages=messages
    )
    return response.choices[0].message.content

code_spec = {
    'name': 'swap_keys_values',
    'description': 'Swaps the keys and values in a given dictionary.',
    'params': {
        'd': 'A dictionary with unique values.'
    },
}

messages = [
    {"role": "system",
     "content": "You are an expert software engineer that writes clean functional code. You always document your functions."},
    {"role": "user", "content": f"Please implement: {json.dumps(code_spec)}"},
    # {"role": "user", "content": f"Can you please shorten the description?"}
]


response = generate_response(messages)
print(response)
print("END")

START
```python
def swap_keys_values(d):
    """Swaps the keys and values in a given dictionary.

    Args:
        d: A dictionary with unique values.

    Returns:
        A new dictionary with keys and values swapped.
    """
    return {v: k for k, v in d.items()}
```
END


In [4]:
# Building a quasi-agent - simple programmatic prompt based agent

# First Prompt

def extract_code_block(response: str) -> str:
    """Extract code block from response"""
    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()
    if code_block.startswith("python"):
        code_block = code_block[6:]

    return code_block

function_description = ""
def get_user_input():
  print("What kind of function would you like to create?")
  print("Eg. Function that generate fibonacci sequence")
  print("Your description: ", end="")
  description = input().strip()
  print("Creating function based on following description: ", description)
  return description

function_description = get_user_input()

messages = [
    { "role": "system", "content": "You are a Python expert helping to develop a function."},
    {"role": "user", "content": f'Write a very small Python function that {function_description}. Output the function in a ```python code block```'}
]

initial_function = extract_code_block(generate_response(messages))
print("=== Initial_function ===")
print(initial_function)

messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
messages.append({"role": "user", "content": "Add comprehensive documentation to this function, including description, parameters, return value, examples, and edge cases. Output the function in a ```python code block```."})

second_function = extract_code_block(generate_response(messages))

print("=== Second_function ===")
print(second_function)

messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
messages.append({"role": "user", "content": "Add unittest test cases for this function, including tests for basic functionality, edge cases, error cases, and various input scenarios. Only generate the testcases. Output the code in a \`\`\`python code block\`\`\`."})

third_function = extract_code_block(generate_response(messages))

print("=== Third_function ===")
print(third_function)

filename = function_description.lower()
filename = "".join(c for c in filename if c.isalnum() or c.isspace())
filename = filename.replace(" ", "_")[:30] + ".py"

with open(filename, "w") as f:
  f.write(second_function + "\n\n" + third_function)


<>:36: SyntaxWarning: invalid escape sequence '\`'
<>:36: SyntaxWarning: invalid escape sequence '\`'
<>:44: SyntaxWarning: invalid escape sequence '\`'
<>:44: SyntaxWarning: invalid escape sequence '\`'
<>:45: SyntaxWarning: invalid escape sequence '\`'
<>:36: SyntaxWarning: invalid escape sequence '\`'
<>:36: SyntaxWarning: invalid escape sequence '\`'
<>:44: SyntaxWarning: invalid escape sequence '\`'
<>:44: SyntaxWarning: invalid escape sequence '\`'
<>:45: SyntaxWarning: invalid escape sequence '\`'
/tmp/ipython-input-1313654286.py:36: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/tmp/ipython-input-1313654286.py:36: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "content": "\`\`\`python\n\n"+initial_function+"\n\n\`\`\`"})
/tmp/ipython-input-1313654286.py:44: SyntaxWarning: invalid escape sequence '\`'
  messages.append({"role": "assistant", "co

What kind of function would you like to create?
Eg. Function that generate fibonacci sequence
Your description: calculate max number in array
Creating function based on following description:  calculate max number in array
initial_function: 

def find_max(arr):
  """
  Finds the maximum number in an array.

  Args:
    arr: A list of numbers.

  Returns:
    The maximum number in the array, or None if the array is empty.
  """
  if not arr:
    return None
  max_num = arr[0]
  for num in arr:
    if num > max_num:
      max_num = num
  return max_num
second_function: 

def find_max(arr):
  """
  Finds the maximum number in an array.

  Description:
    This function iterates through the input array and identifies the largest numerical value.
    It handles empty arrays gracefully by returning None.

  Parameters:
    arr (list of numbers): A list containing numerical values.  The numbers can be integers or floats.

  Returns:
    number: The maximum number found in the array.
    Retur

In [ ]:
#